Обучение случайного леса.

In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline

df = pd.read_csv('prepared_train.csv')
X = df.drop(['text_type'],axis=1)
X['text'] = X['text'].fillna('space')
y  = df['text_type'].replace({'ham':0,'spam':1})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
vectorizer = CountVectorizer()
scaler = StandardScaler()

# создаем трансформер для удобной предобработки данных
preprocessor = ColumnTransformer(
  transformers=[('text', vectorizer, 'text'),('num', scaler, ['spam_symbols', 'not_spam_symbols', 'special_symbols', 'digits','text_len', 'words_count', 'emojis_count'])])

# создаем пайплайн обучения модели
model = Pipeline(steps=[('preprocessor', preprocessor),('classifier', RandomForestClassifier(n_estimators=70))])

model.fit(X_train, y_train)

# предсказать вероятности классов для тестового набора
y_pred_train = model.predict_proba(X_train)[:, 1]
y_pred_test = model.predict_proba(X_test)[:, 1]
results = pd.DataFrame({'Train: ':roc_auc_score(y_train, y_pred_train),'Test: ':roc_auc_score(y_test, y_pred_test)},index=['0'])
results

,Train:,Test:
0,0.999999,0.984759
